<a href="https://colab.research.google.com/github/cristopardo/nasa-space-apps-challenge-2025-meta-analysis/blob/main/Sharks_from_Space_Raw_model_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Simulation and animation of SST & Chlorophyll gradients with shark-like trajectories
# Note: This is a synthetic demo you can later swap to real NASA data (NetCDF) via xarray.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import PillowWriter, FuncAnimation
from pathlib import Path

In [4]:
# -----------------------------
# 1) DOMAIN & TIME
# -----------------------------
rng = np.random.default_rng(7)

# A small region near tropical Pacific (Palmyra-like box, synthetic)
lat_min, lat_max = -3.0, 10.0
lon_min, lon_max = -165.0, -158.0
ny, nx = 90, 120   # keep modest for speed
lats = np.linspace(lat_min, lat_max, ny)
lons = np.linspace(lon_min, lon_max, nx)
Lon, Lat = np.meshgrid(lons, lats)

T = 60  # frames/time-steps for animation

In [5]:
# -----------------------------
# 2) SYNTHETIC ENV FIELDS (time-varying SST, CHL; static depth; dynamic currents/EKE)
# -----------------------------
def moving_gaussian_field(Lon, Lat, t, cx0, cy0, amp=1.0, sigx=1.8, sigy=1.2, speed=(0.03, -0.02)):
    """Create a smooth 'patch' moving with time to emulate eddies/fronts or warm blobs."""
    cx = cx0 + speed[0]*t
    cy = cy0 + speed[1]*t
    return amp * np.exp(-(((Lon - cx)**2)/(2*sigx**2) + ((Lat - cy)**2)/(2*sigy**2)))

def base_sst(Lon, Lat):
    # baseline latitudinal gradient (warmer near equator)
    return 26.0 + 2.0*np.exp(-((Lat-3.5)**2)/8.0)

def base_chl(Lon, Lat):
    # lower background chl with a slight coastal-like gradient
    return 0.12 + 0.05*np.exp(-((Lon+162.0)**2)/10.0)

def streamfunction(Lon, Lat, t):
    # synthetic streamfunction producing rotating eddies
    return 0.8*np.sin(0.5*(Lon + 162.0) + 0.15*t) * np.cos(0.5*(Lat - 4.0) - 0.1*t)

def eke_from_stream(psi):
    # geostrophic-like velocities from streamfunction: u = d(psi)/dy, v = -d(psi)/dx
    dpsi_dy, dpsi_dx = np.gradient(psi, Lat[:,0], Lon[0,:], edge_order=2)
    u = dpsi_dy
    v = -dpsi_dx
    u_ = u - np.nanmean(u)
    v_ = v - np.nanmean(v)
    eke = 0.5*(u_**2 + v_**2)
    return eke

def synthetic_depth(Lon, Lat):
    # simple seamount + slope: shallow near center-top
    z = 2000 - 1500*np.exp(-(((Lon+161.5)**2)/4.5 + ((Lat-6.0)**2)/3.0))  # meters
    z += 600*(Lat - Lat.mean())  # gentle slope
    # Keep within positive depths
    z = np.clip(z, 10, None)
    return z

Depth = synthetic_depth(Lon, Lat)

# Containers over time
SST_ts = np.zeros((T, ny, nx))
CHL_ts = np.zeros((T, ny, nx))
gradSST_ts = np.zeros((T, ny, nx))
gradCHL_ts = np.zeros((T, ny, nx))
EKE_ts = np.zeros((T, ny, nx))
Light_ts = np.zeros(T)  # a simple diurnal factor (0..1)

for t in range(T):
    # SST as base + warm blob + cool blob
    sst = base_sst(Lon, Lat)
    sst += 1.2*moving_gaussian_field(Lon, Lat, t, cx0=-162.2, cy0=5.2, amp=1.0, sigx=1.3, sigy=1.0, speed=(0.015, 0.01))
    sst -= 0.8*moving_gaussian_field(Lon, Lat, t, cx0=-160.2, cy0=2.0, amp=1.0, sigx=1.6, sigy=1.1, speed=(-0.01, -0.008))
    # CHL as base + productive patch
    chl = base_chl(Lon, Lat)
    chl += 0.25*moving_gaussian_field(Lon, Lat, t, cx0=-161.6, cy0=5.8, amp=1.0, sigx=1.1, sigy=1.1, speed=(0.008, -0.006))
    chl = np.clip(chl, 0.01, None)

    # Gradients (magnitude)
    dsst_dy, dsst_dx = np.gradient(sst, lats, lons, edge_order=2)
    dchl_dy, dchl_dx = np.gradient(chl, lats, lons, edge_order=2)
    grad_sst = np.hypot(dsst_dx, dsst_dy)
    grad_chl = np.hypot(dchl_dx, dchl_dy)

    # Currents via streamfunction -> EKE proxy
    psi = streamfunction(Lon, Lat, t)
    eke = eke_from_stream(psi)

    # Light factor (diurnal: 0..1..0 over a day-like cycle)
    Light_ts[t] = 0.5*(1.0 + np.sin(2*np.pi*(t/T)))

    SST_ts[t] = sst
    CHL_ts[t] = chl
    gradSST_ts[t] = grad_sst
    gradCHL_ts[t] = grad_chl
    EKE_ts[t] = eke

In [8]:
# -----------------------------
# 3) HABITAT SUITABILITY MODEL (logistic)
#    S = σ(β0 + β1*SSTz + β2*CHLz + β3*|∇SST|z + β4*EKEz + β5*Depthz + β6*Light)
# -----------------------------
def zscore(arr):
    mu = np.nanmean(arr)
    sd = np.nanstd(arr) + 1e-9
    return (arr - mu)/sd

# Standardize across all space-time
SSTz_ts = zscore(SST_ts)
CHLz_ts = zscore(CHL_ts)
gSSTz_ts = zscore(gradSST_ts)
EKEz_ts = zscore(EKE_ts)
Depthz = zscore(Depth)[None, ...] * np.ones((T, ny, nx))
Lightz_ts = zscore(Light_ts)  # scalar per t

# Model coefficients (tunable; reasonable magnitudes)
beta = {
    "b0": 0.0,
    "b_sst": 1.2,
    "b_chl": 1.8,
    "b_gsst": 0.8,
    "b_eke": 0.6,
    "b_depth": -0.5,  # shallower (lower depth) -> higher suitability (since Depthz: deeper positive/negative depends; choose negative weight)
    "b_light": 0.4
}

def logistic(x):
    return 1.0/(1.0 + np.exp(-x))

S_map = np.zeros((T, ny, nx))
for t in range(T):
    lin = (beta["b0"]
           + beta["b_sst"]*SSTz_ts[t]
           + beta["b_chl"]*CHLz_ts[t]
           + beta["b_gsst"]*gSSTz_ts[t]
           + beta["b_eke"]*EKEz_ts[t]
           + beta["b_depth"]*Depthz[t]
           + beta["b_light"]*Lightz_ts[t])
    S_map[t] = logistic(lin)

In [10]:
# -----------------------------
# 4) iSSF-LIKE TRAJECTORY SIMULATION (N sharks)
# -----------------------------
def latlon_to_idx(lat, lon):
    j = np.searchsorted(lats, lat) - 1
    i = np.searchsorted(lons, lon) - 1
    if (0 <= i < nx) and (0 <= j < ny):
        return j, i
    return None, None

def simulate_tracks(N=6, n_steps=T, step_km=10.0, k_candidates=60):
    tracks = []
    for k in range(N):
        # random start near center
        lat = rng.uniform( (lat_min+lat_max)/2 - 1.5, (lat_min+lat_max)/2 + 1.5 )
        lon = rng.uniform( (lon_min+lon_max)/2 - 1.5, (lon_min+lon_max)/2 + 1.5 )
        hdg = rng.uniform(-np.pi, np.pi)
        for t in range(n_steps):
            # candidate steps with correlation in heading
            cands = []
            for _ in range(k_candidates):
                length = rng.lognormal(mean=np.log(step_km), sigma=0.35)  # km
                theta = hdg + rng.normal(0, 0.7)
                dlat = (length/111.0) * np.sin(theta)
                dlon = (length/(111.0*np.cos(np.radians(lat)))) * np.cos(theta)
                lat2, lon2 = lat + dlat, lon + dlon
                j, i = latlon_to_idx(lat2, lon2)
                if j is None:
                    continue
                # selection weight: exp(g1*S + g2*cos(turn) - g3*length + g4*gSSTz)
                g1, g2, g3, g4 = 2.5, 0.6, 0.05, 0.8
                S_here = S_map[t, j, i]
                grad_here = gSSTz_ts[t, j, i]
                w = np.exp(g1*S_here + g2*np.cos(theta - hdg) - g3*length + g4*grad_here)
                cands.append((w, lat2, lon2, theta))
            if not cands:
                # if no valid candidates, jitter slightly
                lat += rng.normal(0, 0.03); lon += rng.normal(0, 0.03)
            else:
                probs = np.array([c[0] for c in cands])
                probs /= probs.sum()
                idx = rng.choice(len(cands), p=probs)
                _, lat, lon, hdg = cands[idx]
            tracks.append({"id": k, "t": t, "lat": lat, "lon": lon})
    return pd.DataFrame(tracks)

tracks_df = simulate_tracks(N=6, n_steps=T, step_km=8.0, k_candidates=50)

# Guarda en la carpeta actual
csv_path = Path("simulated_tracks.csv")
tracks_df.to_csv(csv_path, index=False)

# Visualiza directamente
print(tracks_df.head(50))

    id   t       lat         lon
0    0   0  2.449695 -161.511570
1    0   1  2.409683 -161.486392
2    0   2  2.378125 -161.412955
3    0   3  2.427955 -161.363038
4    0   4  2.464951 -161.347163
5    0   5  2.528226 -161.307864
6    0   6  2.586515 -161.280544
7    0   7  2.674511 -161.260755
8    0   8  2.713820 -161.262722
9    0   9  2.777653 -161.245470
10   0  10  2.871074 -161.235624
11   0  11  2.905155 -161.237161
12   0  12  2.948936 -161.262289
13   0  13  3.002660 -161.305657
14   0  14  3.088968 -161.357137
15   0  15  3.090765 -161.432986
16   0  16  3.150400 -161.493684
17   0  17  3.196550 -161.473109
18   0  18  3.193919 -161.392927
19   0  19  3.216049 -161.360269
20   0  20  3.279902 -161.420001
21   0  21  3.306233 -161.461747
22   0  22  3.285884 -161.513537
23   0  23  3.263969 -161.557888
24   0  24  3.249269 -161.616962
25   0  25  3.214179 -161.658470
26   0  26  3.122049 -161.643922
27   0  27  3.081051 -161.609570
28   0  28  3.051022 -161.525189
29   0  29

In [11]:
# -----------------------------
# 5) ANIMATIONS: (A) SST gradient + tracks, (B) CHL gradient + tracks
# -----------------------------
def make_animation(bg_stack, bg_label, out_path):
    fig = plt.figure(figsize=(7, 5))
    ax = plt.gca()

    # Initial background
    im = ax.imshow(bg_stack[0], extent=[lon_min, lon_max, lat_min, lat_max],
                   origin='lower', aspect='auto')
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_title(f"{bg_label} + Simulated Shark Routes")

    # Equation text (model shown at all frames)
    eq_text = (
        "S(x,t) = σ(β0 + β1·SSTz + β2·CHLz + β3·|∇SST|z + β4·EKEz + β5·Depthz + β6·Lightz)\n"
        f"β = [{beta['b0']:.1f}, {beta['b_sst']:.1f}, {beta['b_chl']:.1f}, "
        f"{beta['b_gsst']:.1f}, {beta['b_eke']:.1f}, {beta['b_depth']:.1f}, {beta['b_light']:.1f}]"
    )
    model_txt = ax.text(0.01, 0.99, eq_text, transform=ax.transAxes, ha="left", va="top",
                        fontsize=9, bbox=dict(boxstyle="round", alpha=0.3))

    # Time text
    time_txt = ax.text(0.99, 0.01, "", transform=ax.transAxes, ha="right", va="bottom", fontsize=10)

    # Prepare per-shark line/point containers
    N = tracks_df["id"].nunique()
    lines = []
    points = []
    for k in range(N):
        line, = ax.plot([], [], lw=1.2)   # default style (no specific colors)
        pt, = ax.plot([], [], marker="o", ms=4, linestyle="")
        lines.append(line)
        points.append(pt)

    def init():
        im.set_data(bg_stack[0])
        for line, pt in zip(lines, points):
            line.set_data([], [])
            pt.set_data([], [])
        time_txt.set_text("t = 0")
        return [im, model_txt, time_txt, *lines, *points]

    def update(frame):
        im.set_data(bg_stack[frame])
        time_txt.set_text(f"t = {frame+1}/{T}")
        # update each shark trajectory up to current frame
        for k in range(N):
            dfk = tracks_df[ (tracks_df["id"]==k) & (tracks_df["t"]<=frame) ]
            if not dfk.empty:
                lines[k].set_data(dfk["lon"].values, dfk["lat"].values)
                dft = tracks_df[ (tracks_df["id"]==k) & (tracks_df["t"]==frame) ]
                if not dft.empty:
                    points[k].set_data(dft["lon"].values, dft["lat"].values)
        return [im, model_txt, time_txt, *lines, *points]

    anim = FuncAnimation(fig, update, frames=T, init_func=init, blit=True, interval=80)
    writer = PillowWriter(fps=8)
    anim.save(out_path, writer=writer)
    plt.close(fig)

In [13]:
# Build animations
out_sst = "sst_gradient_tracks.gif"
out_chl = "chl_gradient_tracks.gif"
make_animation(gradSST_ts, "SST Gradient", out_sst)
make_animation(gradCHL_ts, "Chlorophyll Gradient", out_chl)

print(out_sst, out_chl, str(csv_path))


sst_gradient_tracks.gif chl_gradient_tracks.gif simulated_tracks.csv


$$
S(x,t) \;=\; \sigma\!\Big(
\beta_0
+ \beta_1 \,\mathrm{SST}_z(x,t)
+ \beta_2 \,\mathrm{CHL}_z(x,t)
+ \beta_3 \,|\nabla \mathrm{SST}|_z(x,t)
+ \beta_4 \,\mathrm{EKE}_z(x,t)
+ \beta_5 \,\mathrm{Depth}_z(x)
+ \beta_6 \,\mathrm{Light}_z(t)
\Big)
$$


**Explicación de cada término:**

- **$S(x,t)$**: probabilidad de idoneidad del hábitat en la posición $x$ y tiempo $t$ (entre 0 y 1).  
- **$\sigma(\cdot)$**: función logística  
  $$
  \sigma(u) = \frac{1}{1+e^{-u}}
  $$  
  transforma la suma lineal en una probabilidad.  

- **$\beta_0$**: intercepto (valor base de la idoneidad).  
- **$\mathrm{SST}_z(x,t)$**: temperatura superficial del mar (z-score). Define rangos térmicos preferidos.  
- **$\mathrm{CHL}_z(x,t)$**: concentración de clorofila-*a* (z-score). Proxy de productividad primaria → más alimento.  
- **$|\nabla \mathrm{SST}|_z(x,t)$**: gradiente de SST (z-score). Marca **frentes térmicos**, zonas donde se concentra el alimento.  
- **$\mathrm{EKE}_z(x,t)$**: energía cinética turbulenta (eddy kinetic energy) estandarizada. Representa remolinos/corrientes que concentran nutrientes.  
- **$\mathrm{Depth}_z(x)$**: batimetría (z-score). Muchas especies limitan sus movimientos a ciertas profundidades.  
- **$\mathrm{Light}_z(t)$**: factor de luz (día/noche o ciclo lunar) estandarizado. Afecta patrones de migración vertical y alimentación.  
- **Coeficientes $\beta_1,\dots,\beta_6$**: pesos que miden la importancia relativa de cada variable ambiental.  